In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
train_data = pd.read_csv('austin_weather.csv')['TempAvgF']
# train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')['Sea_Surface_Temp']
train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:10000]
print(train_data)
train_data.shape

[-0.75772349 -1.6120651  -1.8256505  ...  1.52052079  1.59171592
  1.23574025]


(1319,)

In [9]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,16, 2)
        self.lstm2 = nn.LSTM(16,1)
 
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        
        for X in seq.chunk(seq.size(1),dim=0):
#             print(X)
            tmp, hc1 = self.lstm1(X, hc1)
            X_in, hc2 = self.lstm2(tmp, hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)

In [10]:
seq = Sequence()
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.1)
bs = 32

In [11]:
for epoch in range(50):
    running_loss = 0
    running_error = 0
    num_batches = 0
    for i in range(0, 10000, bs):
        data = train_data[i:i+bs]
        xs = torch.Tensor(data[:-1]).view(-1,1,1)
        ys = torch.Tensor(data[1:])
        
        lstm_out, _ = seq(xs)
        loss = criterion(lstm_out.view(-1), ys)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute some stats:
        
        num_batches+=1
        
        with torch.no_grad():
            running_loss += loss.item()
            error = utils.get_error(lstm_out.view(1, -1), ys.view(1, -1))
            running_error += error 

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    if epoch % 10 == 0 : 
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches

        print('epoch=',epoch, '\t loss=', total_loss, '\t error=', total_error*100 ,'percent')

RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)